In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import time
import csv

In [2]:
from sklearn.model_selection import KFold
def cross_validation(folds, data, labels, error_function, model, **model_args):
    kf = KFold(n_splits=folds, random_state=None, shuffle = True)
    score = []
    y = labels
    if model == LogisticRegression:
        M = model(max_iter = 20**9)
    else:
        M = model()
    predicted_lable = pd.DataFrame(columns = ['prediction'])
    for train_index,test_index in kf.split(data):
        x_training_set = data.iloc[train_index]
        y_training_set = y.iloc[train_index]
        x_test_set = data.iloc[test_index]
        y_test_set = y.iloc[test_index]
        M.fit(np.array(x_training_set), np.array(y_training_set).T[0])
        y_pred = M.predict_proba(np.array(x_test_set))
        y_pred = [x[1] for x in y_pred]
        score.append(error_function(y_test_set,y_pred)) 
        print('.',end=' ')
    average_error = round(sum(score)/folds,4)
    return average_error

In [3]:
def level_1_predictor(t_train,y, model):
    leng = int(len(t_train)/2)
    t_a, t_b= t_train[:leng], t_train[leng:]
    tg_a, tg_b = y[:leng], y[leng:]
    if model == LogisticRegression:
        m1 = model(max_iter = 20**9)
        m2 = model(max_iter = 20**9)
    else:
        m1 = model()
        m2 = model()

    m1.fit(t_a,tg_a)
    tp_b = [i[1] for i in m1.predict_proba(t_b)]
    m2.fit(t_b,tg_b)
    tp_a = [i[1] for i in m2.predict_proba(t_a)]
    tp = [[x] for x in np.hstack((tp_a,tp_b))]
    
    return tp

In [28]:
def retrieve_important_feature(s,m):
    file = pd.read_csv('important_l1.csv')
    feature_name = file.columns
    if m=='G':
        importance = file.iloc[0]
    if m=='L':
        importance = file.iloc[1]
    itr = 0
    important_feature = []
    unimportant_feature = []
    for x in importance:
        if x>s:
            important_feature.append(feature_name[itr])
        else:
            unimportant_feature.append(feature_name[itr])
        itr+=1
    return important_feature, unimportant_feature

In [29]:
def categorize_important_feature(n,m):
    important_feature, unimportant_feature = retrieve_important_feature(n,m)
    delinquency=[]
    spend = []
    payment = []
    balance = []
    risk = []
    std = []
    mean = []
    _25 = []
    _50 = []
    _75 = []
    _min = []
    _max = []

    for x in important_feature:
        if x[0]=='D':
            delinquency.append(x)
        if x[0]=='S':
            spend.append(x)
        if x[0]=='P':
            payment.append(x)
        if x[0]=='B':
            balance.append(x)
        if x[0]=='R':
            risk.append(x)
        if x[-3:]=='std':
            std.append(x)
        elif x[-3:]=='25%':
            _25.append(x)
        elif x[-3:]=='75%':
            _75.append(x)
        elif x[-3:]=='50%':
            _50.append(x)
        elif x[-3:]=='min':
            _min.append(x)
        elif x[-3:]=='max':
            _max.append(x)
        else:
            mean.append(x)  
    feature_sets = {'delinquency':delinquency,'spend':spend,'payment':payment,'balance':balance,'risk':risk,'std':std,
                    'mean':mean,'_25':_25,'_50':_50,'_75':_75,'_min':_min,'_max':_max, 'unimportant_fe':unimportant_feature}
    return feature_sets

In [36]:
def stacker(feature,feature_sets,m='G'):
    column_name = list(feature_sets.keys())
    if m=='L':
        r = pd.read_csv('l1_stacking_model_G.csv')
    if m=='G':
        r = pd.read_csv('l1_stacking_model_L.csv')
    stack_feature = pd.DataFrame(columns = column_name)
    for x in column_name:
        print(x, end='-->')
        best_stacking_model = r[x].value_counts().keys()[0]
        stacking_feature = np.array(feature[feature_sets[x]])
        stack_feature[x] = level_1_predictor(stacking_feature,np_label.T[0], model_dic[best_stacking_model])
        print(best_stacking_model)
    return stack_feature

In [31]:
df= pd.read_csv(r'C:\Users\johnk\OneDrive\Desktop\project\python project\kaggle\amex-default-prediction\new_train\train_9.csv')

In [37]:
feature = df.iloc[:,1:-1]
label = pd.DataFrame(df['target'], columns=['target'])
np_label = np.array(label)
feature_name = feature.columns

In [39]:
feature_sets = categorize_important_feature(1,'L')

In [60]:
models = [LogisticRegression]

In [41]:
model_dic ={}
for x in models:
    model_dic[x.__name__] = x

In [42]:
for x in feature_name:
    feature[x]=feature[x].fillna(feature[x].mean())

In [43]:
stack_feature = stacker(feature, feature_sets)

delinquency-->GradientBoostingClassifier
spend-->RandomForestClassifier
payment-->LogisticRegression
balance-->RandomForestClassifier
risk-->RandomForestClassifier
std-->GradientBoostingClassifier
mean-->LogisticRegression
_25-->GradientBoostingClassifier
_50-->GradientBoostingClassifier
_75-->GradientBoostingClassifier
_min-->GradientBoostingClassifier
_max-->LogisticRegression
unimportant_fe-->GradientBoostingClassifier


In [ ]:
important_feature, unimportant_feature = retrieve_important_feature(1,'G')
feature_u = pd.concat([feature[important_feature],stack_feature], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(feature_u,label, test_size=0.2, random_state=11)
for x in models:
    print(x.__name__,cross_validation(5,X_train, y_train, roc_auc_score,x))

ERROR! Session/line number was not unique in database. History logging moved to new session 298


In [64]:
len(important_feature)

502